## Agenda

* Import packages
* Connect to DB
* load documents into panda dataframe
* Extract profile json
* Flatten json
* Calculate distance
* Save to db

## Import packages

In [41]:
# Install a pip package in the current notebook kernel
#!pip3 install cloudant
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install cloudant
!{sys.executable} -m pip install pixiedust

In [42]:
import numpy as np
import pandas as pd
import time
import json
from pandas.io.json import json_normalize #package for flattening json in pandas df
import pixiedust

In [43]:
import cloudant

## Connect to Cloudant Dbs

In [8]:
from cloudant.client import Cloudant
from cloudant.adapters import Replay429Adapter
#client = Cloudant.iam(cloudant_username, cloudant_apikey, connect=True, adapter=Replay429Adapter(retries=10, initialBackoff=0.01))
client = Cloudant.iam("cloudant_username", 
                      "cloudant_apikey", 
                      connect=True, adapter=Replay429Adapter(retries=10, initialBackoff=0.01))


In [9]:
# Perform client tasks...
session = client.session()
print('Databases: {0}'.format(client.all_dbs()))

Databases: ['c4cprofiles_db', 'community_db', 'edge_db', 'match_db', 'mysample2_db', 'pi_db', 'profiles_db', 'profs_db']


In [10]:
db = client['c4cprofiles_db']

## Load data into pandas dataframe

In [11]:
response = db.all_docs(limit=500, include_docs= True)

# put document bodies into an array
docs = []
for r in response['rows']:
    docs.append(r['doc'])

In [12]:
# create a Pandas dataframe containing the data

df = pd.DataFrame(data=docs)
#display(df.head(7))
df.head(5)

,_id,_rev,ageGroup,id,location,profile,status,user_id,whenCreated
0,1595193640-adult-center_1,1-90ecd23fa686854619c6090daa0e0c4f,adult,1595193640-adult-center_1,center_1,"{'word_count': 7020, 'processed_language': 'en...",active,1595193640-adult-center_1,1595193644273
1,1595193720-adult-center_1,1-9ff4bc6c29183041249974e2a4f41c98,adult,1595193720-adult-center_1,center_1,"{'word_count': 12757, 'processed_language': 'e...",active,1595193720-adult-center_1,1595193725233
2,1595193855-adult-center_1,1-98d7c618d61442d4b6796703a8d02b4d,adult,1595193855-adult-center_1,center_1,"{'word_count': 1668, 'processed_language': 'en...",active,1595193855-adult-center_1,1595193870063
3,1595193915-adult-center_1,1-2d57a126860ddc85b9ef8eff81129eff,adult,1595193915-adult-center_1,center_1,"{'word_count': 516, 'word_count_message': 'The...",active,1595193915-adult-center_1,1595193926261
4,1595193966-adult-center_1,1-97143843b294dbf6f4e9ff342c544c32,adult,1595193966-adult-center_1,center_1,"{'word_count': 685, 'processed_language': 'en'...",active,1595193966-adult-center_1,1595193983397


In [13]:
count_row = df.shape[0]  # gives number of row count
count_col = df.shape[1] 
print('db # rows ' , count_row)
#print('cols ' , count_col)
#print(df.shape)

db # rows  12


In [14]:
grouped = df.groupby([df.ageGroup,df.status])
adults = grouped.get_group(('adult','active'))  #.reset_index()
students = grouped.get_group(('student','active'))
#print(adults['id'])
print("adults " ,adults.shape[0])
print("students " ,students.shape[0])

adults  5
students  7


In [15]:
# create an adult list from a dataframe column 
adult_list = adults['id'].tolist()

j = 0
for i in adult_list: 
    print('row:', j , '--id:', adult_list[j]) # printing ids
    j+=1

row: 0 --id: 1595193640-adult-center_1
row: 1 --id: 1595193720-adult-center_1
row: 2 --id: 1595193855-adult-center_1
row: 3 --id: 1595193915-adult-center_1
row: 4 --id: 1595193966-adult-center_1


In [16]:
# create a student list from a dataframe column 
student_list = students['id'].tolist()

j = 0
for i in student_list: 
    print('row:', j , '--id:', student_list[j]) # printing ids 
    j+=1

row: 0 --id: 1595194028-student-HS_1
row: 1 --id: 1595194177-student-HS_1
row: 2 --id: 1595194242-student-HS_1
row: 3 --id: 1595194312-student-HS_1
row: 4 --id: 1595194359-student-HS_1
row: 5 --id: 1595194416-student-HS_1
row: 6 --id: 1595194488-student-HS_1


## Extract profile data into json

```json
[
  {
    "word_count": 31210,
    "processed_language": "en",
    "personality": [
      {
        "trait_id": "big5_openness",
        "name": "Openness",
        "category": "personality",
        "percentile": 0.41082833860271567,
        "significant": true,
        "children": [
          {
            "trait_id": "facet_adventurousness",
            "name": "Adventurousness",
            "category": "personality",
            "percentile": 0.9928832722907611,
            "significant": true
          },
          {
            "trait_id": "facet_artistic_interests",
            "name": "Artistic interests",
            "category": "personality",
            "percentile": 0.617785710647723,
            "significant": true
          },
          {
            "trait_id": "facet_emotionality",
            "name": "Emotionality",
            "category": "personality",
            "percentile": 0.32497110283541053,
            "significant": true
          },
          {
            "trait_id": "facet_imagination",
            "name": "Imagination",
            "category": "personality",
            "percentile": 0.3500777779046056,
            "significant": true
          },
          {
            "trait_id": "facet_intellect",
            "name": "Intellect",
            "category": "personality",
            "percentile": 0.6927187072564286,
            "significant": true
          },
          {
            "trait_id": "facet_liberalism",
            "name": "Authority-challenging",
            "category": "personality",
            "percentile": 0.7654294245462194,
            "significant": true
          }
        ]
      },
```

In [17]:
adults_profile = json_normalize(adults['profile'])
adults_profile.head(5)

,consumption_preferences,needs,personality,processed_language,values,warnings,word_count,word_count_message
0,[{'consumption_preference_category_id': 'consu...,"[{'trait_id': 'need_challenge', 'name': 'Chall...","[{'trait_id': 'big5_openness', 'name': 'Openne...",en,"[{'trait_id': 'value_conservation', 'name': 'C...",[],7020,NaN
1,[{'consumption_preference_category_id': 'consu...,"[{'trait_id': 'need_challenge', 'name': 'Chall...","[{'trait_id': 'big5_openness', 'name': 'Openne...",en,"[{'trait_id': 'value_conservation', 'name': 'C...",[],12757,NaN
2,[{'consumption_preference_category_id': 'consu...,"[{'trait_id': 'need_challenge', 'name': 'Chall...","[{'trait_id': 'big5_openness', 'name': 'Openne...",en,"[{'trait_id': 'value_conservation', 'name': 'C...",[],1668,NaN
3,[{'consumption_preference_category_id': 'consu...,"[{'trait_id': 'need_challenge', 'name': 'Chall...","[{'trait_id': 'big5_openness', 'name': 'Openne...",en,"[{'trait_id': 'value_conservation', 'name': 'C...","[{'warning_id': 'WORD_COUNT_MESSAGE', 'message...",516,There were 516 words in the input. We need a m...
4,[{'consumption_preference_category_id': 'consu...,"[{'trait_id': 'need_challenge', 'name': 'Chall...","[{'trait_id': 'big5_openness', 'name': 'Openne...",en,"[{'trait_id': 'value_conservation', 'name': 'C...",[],685,NaN


In [18]:
students_profile = json_normalize(students['profile'])
students_profile.head(5)

,consumption_preferences,needs,personality,processed_language,values,warnings,word_count
0,[{'consumption_preference_category_id': 'consu...,"[{'trait_id': 'need_challenge', 'name': 'Chall...","[{'trait_id': 'big5_openness', 'name': 'Openne...",en,"[{'trait_id': 'value_conservation', 'name': 'C...",[],2327
1,[{'consumption_preference_category_id': 'consu...,"[{'trait_id': 'need_challenge', 'name': 'Chall...","[{'trait_id': 'big5_openness', 'name': 'Openne...",en,"[{'trait_id': 'value_conservation', 'name': 'C...",[],9544
2,[{'consumption_preference_category_id': 'consu...,"[{'trait_id': 'need_challenge', 'name': 'Chall...","[{'trait_id': 'big5_openness', 'name': 'Openne...",en,"[{'trait_id': 'value_conservation', 'name': 'C...",[],1803
3,[{'consumption_preference_category_id': 'consu...,"[{'trait_id': 'need_challenge', 'name': 'Chall...","[{'trait_id': 'big5_openness', 'name': 'Openne...",en,"[{'trait_id': 'value_conservation', 'name': 'C...",[],1804
4,[{'consumption_preference_category_id': 'consu...,"[{'trait_id': 'need_challenge', 'name': 'Chall...","[{'trait_id': 'big5_openness', 'name': 'Openne...",en,"[{'trait_id': 'value_conservation', 'name': 'C...",[],1224


In [19]:
adult_personality_data = json_normalize(data=adults['profile'], record_path='personality'
                            ,errors='ignore')
adult_personality_data.head(6) # 5 + 1 new set

,category,children,name,percentile,significant,trait_id
0,personality,"[{'trait_id': 'facet_adventurousness', 'name':...",Openness,0.943621,True,big5_openness
1,personality,"[{'trait_id': 'facet_achievement_striving', 'n...",Conscientiousness,0.871738,True,big5_conscientiousness
2,personality,"[{'trait_id': 'facet_activity_level', 'name': ...",Extraversion,0.633776,True,big5_extraversion
3,personality,"[{'trait_id': 'facet_altruism', 'name': 'Altru...",Agreeableness,0.216472,True,big5_agreeableness
4,personality,"[{'trait_id': 'facet_anger', 'name': 'Fiery', ...",Emotional range,0.007168,True,big5_neuroticism
5,personality,"[{'trait_id': 'facet_adventurousness', 'name':...",Openness,0.999574,True,big5_openness


In [20]:
student_personality_data = json_normalize(data=students['profile'], record_path='personality'
                            ,errors='ignore')
student_personality_data.head(6) # 5 + 1 new set

,category,children,name,percentile,significant,trait_id
0,personality,"[{'trait_id': 'facet_adventurousness', 'name':...",Openness,0.985286,True,big5_openness
1,personality,"[{'trait_id': 'facet_achievement_striving', 'n...",Conscientiousness,0.882516,True,big5_conscientiousness
2,personality,"[{'trait_id': 'facet_activity_level', 'name': ...",Extraversion,0.553078,True,big5_extraversion
3,personality,"[{'trait_id': 'facet_altruism', 'name': 'Altru...",Agreeableness,0.242844,True,big5_agreeableness
4,personality,"[{'trait_id': 'facet_anger', 'name': 'Fiery', ...",Emotional range,0.252514,True,big5_neuroticism
5,personality,"[{'trait_id': 'facet_adventurousness', 'name':...",Openness,0.998691,True,big5_openness


In [21]:
adult_personality_children_data = json_normalize(data=adults['profile'], record_path=['personality', 'children']
                              ,errors='ignore') #, record_prefix='_')
adult_personality_children_data.head(31) #30 + 1 new set
# 6 children / each personality dimension : [ Openness, Conscientiousness, Extraversion, Agreeableness, Emotional-range ]

,category,name,percentile,significant,trait_id
0,personality,Adventurousness,0.872805,True,facet_adventurousness
1,personality,Artistic interests,0.418483,True,facet_artistic_interests
2,personality,Emotionality,0.158366,True,facet_emotionality
3,personality,Imagination,0.044812,True,facet_imagination
4,personality,Intellect,0.996393,True,facet_intellect
5,personality,Authority-challenging,0.883534,True,facet_liberalism
6,personality,Achievement striving,0.898745,True,facet_achievement_striving
7,personality,Cautiousness,0.971289,True,facet_cautiousness
8,personality,Dutifulness,0.813399,True,facet_dutifulness
9,personality,Orderliness,0.287742,True,facet_orderliness


In [22]:
student_personality_children_data = json_normalize(data=students['profile'], record_path=['personality', 'children']
                              ,errors='ignore')
student_personality_children_data.head(6) #6 children / 1 personality name
# 6 children / each personality name : [ Openness, Conscientiousness, Extraversion, Agreeableness, Emotional-range ]

,category,name,percentile,significant,trait_id
0,personality,Adventurousness,0.934679,True,facet_adventurousness
1,personality,Artistic interests,0.982239,True,facet_artistic_interests
2,personality,Emotionality,0.887707,True,facet_emotionality
3,personality,Imagination,0.280627,True,facet_imagination
4,personality,Intellect,0.970637,True,facet_intellect
5,personality,Authority-challenging,0.983536,True,facet_liberalism


In [23]:
adult_needs_data = json_normalize(data=adults['profile'], record_path='needs'
                            ,errors='ignore')
adult_needs_data.head(13) # 12 + 1 new set

,category,name,percentile,significant,trait_id
0,needs,Challenge,0.146272,True,need_challenge
1,needs,Closeness,0.022737,True,need_closeness
2,needs,Curiosity,0.367466,True,need_curiosity
3,needs,Excitement,0.040899,True,need_excitement
4,needs,Harmony,0.012459,True,need_harmony
5,needs,Ideal,0.071551,True,need_ideal
6,needs,Liberty,0.029330,True,need_liberty
7,needs,Love,0.010630,True,need_love
8,needs,Practicality,0.112633,True,need_practicality
9,needs,Self-expression,0.006772,True,need_self_expression


In [24]:
student_needs_data = json_normalize(data=students['profile'], record_path='needs'
                            ,errors='ignore')
student_needs_data.head(13) # 12 + 1 new set

,category,name,percentile,significant,trait_id
0,needs,Challenge,0.039853,True,need_challenge
1,needs,Closeness,0.388361,True,need_closeness
2,needs,Curiosity,0.893457,True,need_curiosity
3,needs,Excitement,0.025298,True,need_excitement
4,needs,Harmony,0.115464,True,need_harmony
5,needs,Ideal,0.122216,True,need_ideal
6,needs,Liberty,0.054097,True,need_liberty
7,needs,Love,0.030652,True,need_love
8,needs,Practicality,0.002545,True,need_practicality
9,needs,Self-expression,0.235123,True,need_self_expression


In [25]:
adult_values_data = json_normalize(data=adults['profile'], record_path='values'
                            ,errors='ignore')
adult_values_data.head(6) # 5 + 1 new set

,category,name,percentile,significant,trait_id
0,values,Conservation,0.098071,True,value_conservation
1,values,Openness to change,0.142036,True,value_openness_to_change
2,values,Hedonism,0.011761,True,value_hedonism
3,values,Self-enhancement,0.004195,True,value_self_enhancement
4,values,Self-transcendence,0.058710,True,value_self_transcendence
5,values,Conservation,0.013431,True,value_conservation


In [26]:
student_values_data = json_normalize(data=students['profile'], record_path='values'
                            ,errors='ignore')
student_values_data.head(6) # 5 + 1 new set

,category,name,percentile,significant,trait_id
0,values,Conservation,0.235003,True,value_conservation
1,values,Openness to change,0.604861,True,value_openness_to_change
2,values,Hedonism,0.028378,True,value_hedonism
3,values,Self-enhancement,0.029560,True,value_self_enhancement
4,values,Self-transcendence,0.497447,True,value_self_transcendence
5,values,Conservation,0.182441,True,value_conservation


## Flatten json

In [ ]:
i = 1
j = 0

adult_dict ={}
big5_list = []

for index,row in adult_personality_data.iterrows():
    #print('i:' , i , "=", row[1] , '-- with %:', row[3]) # personality name & percentile
    big5_list.append(row[3])
    if  (i%5 == 0):
        #print('row:' , j , big5_list)
        big5_string = ', '.join( str(element) for element in big5_list )
        adult_dict[adult_list[j]] = big5_string
        big5_list.clear()
        j += 1
    i += 1

print('adults dictionary')
display(adult_dict)

1595193640-adult-center_1,1595193720-adult-center_1,1595193855-adult-center_1,1595193915-adult-center_1,1595193966-adult-center_1
"0.9436207034687292, 0.8717383918743111, 0.633776377853311, 0.2164721952851401, 0.007168210746788084","0.9995738169166637, 0.7438435478885108, 0.034986090111708223, 0.009571779942115222, 0.4181880914505913","0.9971829354818535, 0.9663922549625675, 0.12278176991230494, 0.30998421817043215, 0.07119296201918163","0.994722313334144, 0.9750953055170224, 0.336284698244183, 0.1439816067786987, 0.8671033130605783","0.9708641324648897, 0.908291791691866, 0.2721777057414253, 0.19613621232900902, 0.31805894886709896"


In [ ]:
i = 1
j = 0
student_dict ={}
big5_list = []

for index,row in student_personality_data.iterrows():
    #print('i:' , i , "=", row[2] , '-- with %:', row[3]) # name & percentile
    big5_list.append(row[3])
    if  (i%5 == 0):
        #print('row:' , j , big5_list)
        big5_string = ', '.join(str(e) for e in big5_list)
        student_dict[student_list[j]] = big5_string
        big5_list.clear()
        j += 1
    i += 1
    
print('students dictionary')
display(student_dict)

1595194028-student-HS_1,1595194177-student-HS_1,1595194242-student-HS_1,1595194312-student-HS_1,1595194359-student-HS_1,1595194416-student-HS_1,1595194488-student-HS_1
"0.9852858790368701, 0.882516161840654, 0.5530784883478886, 0.24284403572530205, 0.25251384331279647","0.9986905679425517, 0.9276462090045288, 0.22381284331948786, 0.2882666223068032, 0.14597459453939615","0.9778626336427217, 0.9542770630136106, 0.09695829282398238, 0.15909690427413015, 0.2546784674387036","0.9997429304751213, 0.9456492777819139, 0.6392428856393143, 0.10297593838695623, 0.15217464234970873","0.9812831366150581, 0.8760911545307675, 0.516607472214709, 0.8796990447356727, 0.4069799370414776","0.9929968757156731, 0.7652454231826358, 0.12659872255243043, 0.5152800354043799, 0.23965176833492668","0.98746354951764, 0.5802219330670527, 0.2807769308456634, 0.6121673715919668, 0.36565122459990756"


## Calculate distance

In [33]:
a_s_list = []
adult_student_match_df = pd.DataFrame()

for a_id in adult_dict:
    a = np.array(adult_dict[a_id].split(', ')) #strings into array
    x = a.astype(np.float)  # cast array elements to float
    
    for s_id in student_dict:
        b = np.array(student_dict[s_id].split(', '))
        y = b.astype(np.float)
        
        dist = np.linalg.norm(x-y)  ## distance function from numpy
        
        a_s_list.append(a_id)
        a_s_list.append(s_id)
        a_s_list.append(dist)
        
        numberElements = len(a_s_list)
        newRow = pd.DataFrame(np.array(a_s_list).reshape(1,numberElements), columns=['adult_id','student_id','distance'])
        adult_student_match_df = adult_student_match_df.append(newRow, ignore_index=True)
        a_s_list.clear()

adult_student_match_df.head(8)  # 7  + 1 combinations of 1 adult & 7 students

,adult_id,student_id,distance
0,1595193640-adult-center_1,1595194028-student-HS_1,0.2631620222194246
1,1595193640-adult-center_1,1595194177-student-HS_1,0.44570183482889764
2,1595193640-adult-center_1,1595194242-student-HS_1,0.6005930815588316
3,1595193640-adult-center_1,1595194312-student-HS_1,0.20627814150629042
4,1595193640-adult-center_1,1595194359-student-HS_1,0.7841461778116451
5,1595193640-adult-center_1,1595194416-student-HS_1,0.6436945414314055
6,1595193640-adult-center_1,1595194488-student-HS_1,0.7046966682669955
7,1595193720-adult-center_1,1595194028-student-HS_1,0.6080443026239899


In [34]:
sort_by_adult_student_distance = adult_student_match_df.sort_values(['adult_id','distance'])
print(sort_by_adult_student_distance)

                     adult_id               student_id             distance
3   1595193640-adult-center_1  1595194312-student-HS_1  0.20627814150629042
0   1595193640-adult-center_1  1595194028-student-HS_1   0.2631620222194246
1   1595193640-adult-center_1  1595194177-student-HS_1  0.44570183482889764
2   1595193640-adult-center_1  1595194242-student-HS_1   0.6005930815588316
5   1595193640-adult-center_1  1595194416-student-HS_1   0.6436945414314055
6   1595193640-adult-center_1  1595194488-student-HS_1   0.7046966682669955
4   1595193640-adult-center_1  1595194359-student-HS_1   0.7841461778116451
9   1595193720-adult-center_1  1595194242-student-HS_1   0.3125497618413505
8   1595193720-adult-center_1  1595194177-student-HS_1   0.4703304627072026
12  1595193720-adult-center_1  1595194416-student-HS_1   0.5445275283755024
7   1595193720-adult-center_1  1595194028-student-HS_1   0.6080443026239899
13  1595193720-adult-center_1  1595194488-student-HS_1   0.6732112582661427
10  15951937

## Remove low scores

In [35]:
# Get names of indexes for which column distance has value >= 50%
indexNames = sort_by_adult_student_distance[ sort_by_adult_student_distance['distance'].astype(np.float) >= 0.5 ].index
# Delete these row indexes from dataFrame
sort_by_adult_student_distance.drop(indexNames , inplace=True)
print(sort_by_adult_student_distance)

                     adult_id               student_id             distance
3   1595193640-adult-center_1  1595194312-student-HS_1  0.20627814150629042
0   1595193640-adult-center_1  1595194028-student-HS_1   0.2631620222194246
1   1595193640-adult-center_1  1595194177-student-HS_1  0.44570183482889764
9   1595193720-adult-center_1  1595194242-student-HS_1   0.3125497618413505
8   1595193720-adult-center_1  1595194177-student-HS_1   0.4703304627072026
15  1595193855-adult-center_1  1595194177-student-HS_1  0.13332199128167455
16  1595193855-adult-center_1  1595194242-student-HS_1  0.24004336296166354
19  1595193855-adult-center_1  1595194416-student-HS_1   0.3331919599240164
14  1595193855-adult-center_1  1595194028-student-HS_1   0.4792880876234476
30  1595193966-adult-center_1  1595194242-student-HS_1  0.19558748075396679
29  1595193966-adult-center_1  1595194177-student-HS_1  0.20393406126750122
28  1595193966-adult-center_1  1595194028-student-HS_1   0.2936926985232705
33  15951939

## Write results to Database

In [36]:
match_db = client['match_db']

In [37]:
i=0
for index,row in sort_by_adult_student_distance.iterrows():
    print('Saving...:' , i , "adult:", row[0] , 'student:', row[1] , 'distance:', row[2])
    data = {
        '_id': row[0], # Setting _id is optional
        'adult_id': row[0],
        'student_id': row[1],
        'distance': row[2],
        'date': time.time()
    }
    #call cloudant db and save row
    # # Create a document using the Database API
    #doc = match_db.create_document(data)
    i+= 1

Saving...: 0 adult: 1595193640-adult-center_1 student: 1595194312-student-HS_1 distance: 0.20627814150629042
Saving...: 1 adult: 1595193640-adult-center_1 student: 1595194028-student-HS_1 distance: 0.2631620222194246
Saving...: 2 adult: 1595193640-adult-center_1 student: 1595194177-student-HS_1 distance: 0.44570183482889764
Saving...: 3 adult: 1595193720-adult-center_1 student: 1595194242-student-HS_1 distance: 0.3125497618413505
Saving...: 4 adult: 1595193720-adult-center_1 student: 1595194177-student-HS_1 distance: 0.4703304627072026
Saving...: 5 adult: 1595193855-adult-center_1 student: 1595194177-student-HS_1 distance: 0.13332199128167455
Saving...: 6 adult: 1595193855-adult-center_1 student: 1595194242-student-HS_1 distance: 0.24004336296166354
Saving...: 7 adult: 1595193855-adult-center_1 student: 1595194416-student-HS_1 distance: 0.3331919599240164
Saving...: 8 adult: 1595193855-adult-center_1 student: 1595194028-student-HS_1 distance: 0.4792880876234476
Saving...: 9 adult: 1595

In [38]:
# Disconnect from the server
client.disconnect()
print('done')

done


In [39]:
display(sort_by_adult_student_distance)

adult_id,student_id,distance
1595193640-adult-center_1,1595194312-student-HS_1,0.20627814150629042
1595193640-adult-center_1,1595194028-student-HS_1,0.2631620222194246
1595193640-adult-center_1,1595194177-student-HS_1,0.44570183482889764
1595193720-adult-center_1,1595194242-student-HS_1,0.3125497618413505
1595193720-adult-center_1,1595194177-student-HS_1,0.4703304627072026
1595193855-adult-center_1,1595194177-student-HS_1,0.13332199128167455
1595193855-adult-center_1,1595194242-student-HS_1,0.24004336296166354
1595193855-adult-center_1,1595194416-student-HS_1,0.3331919599240164
1595193855-adult-center_1,1595194028-student-HS_1,0.4792880876234476
1595193966-adult-center_1,1595194242-student-HS_1,0.19558748075396679
